## Time Series Forecast using Vertex Forecast

Time Series Dataset을 이용한 Forecast 모델을 생성하는 것은, 매우 잘 알려져 있는 내용이다. 

해당 Notebook에서는 어떻게 빠르고 편하게, BigQuery에 데이터셋을 입력하고, 이를 이용하여 Vertex AI Platform에서 AUTOML을 적용하여
Forecast모델을 생성하고, 이를 활용하는지를 정리해 보았다. 

먼저, 기본적인 Notebook내용은 해당 Link(https://www.kaggle.com/code/piergiacomofonseca/smart-home-iot-eda-arimas-lstm-and-more/notebook)를 참고하여 구성하였다.


### Goal

1. [Data Loading](#data-loading) - Vertex Forecast를 이용하기 위해서, Panda Dataframe을 BigQuery로 Loading하는지 확인해 본다.
2. [Dataset configuration](#) - BigQuery에서 어떻게 DataSet을 구성하는지 확인해 본다. 또한 해당 Dataset을 통하여 Training 작업을 수행한다.
3. Vertex Forecast 모델을 이용한 Prediction 값을 생성해 보고 이를 실제 결과치와 비교해 본다. (작성 중)
4. Vertex Forecast 모델의 Prediction 값에 대한 Confidence 구간을 생성해 본다. (작성 중)



### Data Loading

샘플 데이터를 BigQuery로 Load하는 부분입니다. 
이미 BigQuery에 필요한 자료가 모두 포함되어 있다면 하기에 나온 Cell들을 수행할 필요가 없으며, [Dataset configuration](#dataset-configuration) 구성 항목으로 바로 진행한다.

In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_rows', None)
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from datetime import datetime
import warnings
import os
warnings.filterwarnings("ignore")

service account credential file을 이용할 경우, credentail_file_path를 설정해 준다. 만약 notebook instance의 default profile을 이용할 경우, "" 값으로 대치한다.

알맞는 project_id, table_name 을 설정한다. table_name에는 schema name(dataset name)을 포함하여 설정한다.

하기 내용을 직접 수정하거나 아니면 같은 소스 폴더에 있는 environ.env 파일을 구성하여 활용한다.

In [ ]:

! source ./environ.env

project_id = os.environ['project_id']
table_name = os.environ['table_name']
credential_file_path = os.environ['credential_file_path']

HomeC.csv 파일은 Kaggle 데이터셋을 가지고와서 구성하여야 한다. 
위에 있는 Kaggle Link를 통하여, HomeC.zip 파일을 가지고 온 후, unzip HomeC.zip 파일로 HomeC.csv 파일을 생성하고, 

! mkdir ../data

를 수행하여 data folder를 생성하고, 파일을 이동한 후에 아래 Cell을 실행한다.

In [ ]:
#! mkdir ../data
#! mv <<HomeC.CSV file path>> ../data

data = pd.read_csv("../data/HomeC.csv",low_memory=False)
data = data[:-1] #delete last row  (NaNs)
data.info()

하기 내용은, Time Series Column을 Unix Epoch Time에서 ISO 표준 포멧으로 변경해 준다.

In [ ]:
data['time_ts'] = pd.to_datetime(data['time'], unit='s')
data['time_ts'] = pd.DatetimeIndex(pd.date_range('2016-01-01 05:00', periods=len(data),  freq='min'))
data.head(2)

하기 코드는, 샘플 데이터셋에 포함되어 있는 불필요한 Field Name을 정리하고, 일부 필드값을 통합해 줍니다. 
샘플 데이터에 따라 달라질 수 있습니다. 

또한 데이터프레임의 컬럼명이 향후 BigQuery Table의 Column명으로 지정되기 때문에, DataFrame의 컬럼명에 특수문자가 포함되는 것을 제거해야 합니다.

(여러분들이 실제 데이터셋을 가지고 있다면, 밑에 코드는 큰 의미가 없습니다.)

In [ ]:
#Delete '[kW]' in columns name, sum similar consumtions and delete 'summary' column
data.columns = [i.replace(' [kW]', '').replace(' ', '_') for i in data.columns]
data['Furnace'] = data[['Furnace_1','Furnace_2']].sum(axis=1)
data['Kitchen'] = data[['Kitchen_12','Kitchen_14','Kitchen_38']].sum(axis=1) #We could also use the mean 
data.drop(['Furnace_1','Furnace_2','Kitchen_12','Kitchen_14','Kitchen_38','icon','summary', 'time'], axis=1, inplace=True)

#Replace invalid values in column 'cloudCover' with backfill method
data['cloudCover'].replace(['cloudCover'], method='bfill', inplace=True)
data['cloudCover'] = data['cloudCover'].astype('float')

#Reorder columns
data = data[['time_ts', 'use', 'gen', 'House_overall', 'Dishwasher', 'Home_office', 'Fridge', 'Wine_cellar', 'Garage_door', 'Barn',
             'Well', 'Microwave', 'Living_room', 'Furnace', 'Kitchen', 'Solar', 'temperature', 'humidity', 'visibility', 
             'apparentTemperature', 'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity', 
             'dewPoint', 'precipProbability']]
data.head(2)

In [ ]:
import os
import google.auth
if not credential_file_path and credential_file_path != "":
  os.environ['GOOGLE_APPLICATION_CREDENTIALS']=credential_file_path
credentials, project_id = google.auth.default()
data.to_gbq(destination_table=table_name
  ,project_id=project_id
  ,credentials=credentials
  )

위 내용까지 수행하면, BigQuery에 샘플 데이터가 Loading됩니다. (테이블 자동 생성)

정상적으로 Loading되어 있는지 확인해 보겠습니다. 

In [ ]:
test_data = pd.read_gbq("select * from {project_id}.{table_name} limit 1000".format(project_id=project_id, table_name=table_name))
test_data.head(2)
test_data.info()

In [ ]:
dfm = pd.melt(test_data,id_vars='time_ts',value_vars=['use', 'gen', 'House_overall', 'Dishwasher', 'Home_office', 'Fridge', 'Wine_cellar', 'Garage_door', 'Barn',
             'Well', 'Microwave', 'Living_room', 'Furnace', 'Kitchen', 'Solar', 'temperature', 'humidity', 'visibility', 
             'apparentTemperature', 'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity', 
             'dewPoint', 'precipProbability'])
dfm

In [ ]:
#data = data.set_index('time_ts')


### Dataset configuration

BigQuery에서 AutoML을 이용한 Model을 간단하게 생성하기 위해서는, 사전에 Dataset Configuration을 진행해야 한다.
Dataset은 GCS, BigQuery에 있는 소스데이터에 대한 Wrapping 역할을 해준다. 

BigQuery에서 Multi Variant DataSet Format은 [Narrow Data Format](https://en.wikipedia.org/wiki/Wide_and_narrow_data)을 따라야 한다. 

이를 위해서 전처리 작업을 수행하여야 하는데, 주의할 사항이 있다. 

먼저 독립 변수를 추출하여야 한다. 예를 들어, "CloudCoverage(구름낀정도)"의 경우, 전력생산/전력사용량에는 영향을 주지만, 집안의 전력생산의 변화가 역으로 CloudCoverage에 영향을 주지 못한다. 
즉, 외생변수로서 입력 역할을 하여야 하며, 이 부분을 Forecasting하는 것은 일단 제외할 수 있다. 
하지만, 시계열 예측에서는 "Cloud Coverage"와 같은 외생 변수조차 그 전일의 데이터에 영향을 받을 수 밖에 없기 때문에, 일반 컬럼으로 분류하지 않고 마찬가지로 Variable/Value의 Narrow Data Format으로 지정한다.

공휴일 정보는 외생 독립변수로 취급할 수 있기 때문에 일반 컬럼으로 분류할 수 있다. 

이 부분을 정리하면 아래와 같은 형태의 자료 Set이 될 것이다. (일단, Holiday 정보는 BigQuery에서 JOIN을 통해서 쉽게 추가할 수 있으므로 DataFrame에서는 holiday_yn column이 존재하지 않는다.)

        time_ts  |  holiday_yn  |  variable  |  value

data가 가지고 있는 컬럼을 보면, 
'use', 'gen', 'House_overall', 'Dishwasher', 'Home_office', 'Fridge', 'Wine_cellar', 'Garage_door', 'Barn',
'Well', 'Microwave', 'Living_room', 'Furnace', 'Kitchen', 'Solar', 'temperature', 'humidity', 'visibility', 
'apparentTemperature', 'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity', 
'dewPoint', 'precipProbability'

가 있으므로, 이 컬럼들을 모두 Narrow Data Format으로 변경해 본다.

In [ ]:
narrow_format_table_name = "{table_name}_nf".format(table_name=table_name)

In [ ]:
wide_data = pd.read_gbq("select * from {project_id}.{table_name}".format(project_id=project_id, table_name=table_name))
narrow_data = pd.melt(test_data,id_vars='time_ts',value_vars=['use', 'gen', 'House_overall', 'Dishwasher', 'Home_office', 'Fridge', 'Wine_cellar', 'Garage_door', 'Barn',
             'Well', 'Microwave', 'Living_room', 'Furnace', 'Kitchen', 'Solar', 'temperature', 'humidity', 'visibility', 
             'apparentTemperature', 'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity', 
             'dewPoint', 'precipProbability'])


위에서 나온 Narrow Data Format으로 되어 있는 DataFrame 정보가 향후에 Vertext Dataset으로 활용될 것이다. 

In [ ]:
narrow_data.head(100)

In [ ]:
narrow_data.to_gbq(destination_table=narrow_format_table_name
  ,project_id=project_id
  ,credentials=credentials
  )

이후는 Vertex AI Platform 에서 진행하면 된다. 

Vertex AI Platform에서 [Datasets configuration](https://docs.google.com/document/d/10XevFHgUdTCwOdBUDt_cLYN-anTZQf81sKUlIa7ughk/edit?usp=sharing)

해당 문서를 참고해서 진행하면, DataSet 구성과 Training이 동시에 진행된다.